In [1]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import ast
pd.options.display.max_columns=None

In [2]:
attack=pd.read_csv('../ETL/datos/attacks_limpieza_completa.csv', index_col=0)
df=pd.read_csv('../ETL/datos/df_API_paises.csv', index_col=0)

In [3]:
class Concat_Paises():
    def __init__ (self,producto):
        self.producto=producto
        
    def paises(self):

        df_concat=pd.DataFrame()
        dict={'USA':[39.783730,-100.445882],'Australia':[-24.77610,134.755], 'South Africa':[-28.8166236,24.991639],\
            'New Zealand':[-41.5000831,172.8344077],'Papua New Guinea':[-5.6816069,144.2489081] }
            
        for k,v in dict.items():
                
            url = f'http://www.7timer.info/bin/api.pl?lon=-{v[1]}&lat={v[0]}&product={self.producto}&output=json'

            response = requests.get(url)
            df=pd.json_normalize(response.json()["dataseries"])
            df["latitud"] = v[0]
            df["longitud"] = v[1]
            df['paises']= k
            df_concat=pd.concat([df_concat,df], axis=0, ignore_index=True)

        return df_concat

In [4]:

class Limpiar_dfs():
    
    def __init__ (self,dataframe1,dataframe2):
        self.dataframe1=dataframe1
        self.dataframe2=dataframe2
        #self.df_left=df_left
        #self.df_rigth=df_rigth
    
    def limpiar_attack (self):
        df_attack=self.dataframe1[(self.dataframe1["country"]=="usa") | (self.dataframe1["country"]=="australia")|\
            (self.dataframe1["country"]=="south africa") | (self.dataframe1["country"]=="papua new guinea")|\
            (self.dataframe1["country"]=="new zealand")]
        df_attack.rename(columns={'country':'id_paises'}, inplace=True)
        df_final=df_attack.groupby('id_paises')['year','age','latitud','longitud','fatal_N'].agg(['mean'])
        df_final=df_final.reset_index()
        return df_final
    
    def limpieza_clima(self):
        #Limpiamos la columna de rh_profile primero a series y luego a columnas en el bbucle for
        self.dataframe2['rh_profile']=self.dataframe2['rh_profile'].apply(ast.literal_eval)
        rh = self.dataframe2['rh_profile'].apply(pd.Series)

        for i in range(len(rh.columns)):
            rh_col0=rh[i].apply(pd.Series)
            nombre="rh"+ rh_col0["layer"][115]
            valores= list(rh_col0["rh"])
            self.dataframe2.insert(i, nombre, valores)
        #Eliminamos la columna que ya no necesitamos
        self.dataframe2=self.dataframe2.drop(['rh_profile'], axis=1)
        
        #Hacemos lo mismo para la columnas wind_profile
        self.dataframe2['wind_profile']=self.dataframe2['wind_profile'].apply(ast.literal_eval)
        wind = self.dataframe2['wind_profile'].apply(pd.Series)

        for i in range(len(wind.columns)):
            wind_col0=wind[i].apply(pd.Series)
            nombre="wind_"+ wind_col0["layer"][1]
            valores= list(wind_col0["direction"])
            self.dataframe2.insert(i, nombre, valores)
            
        self.dataframe2=self.dataframe2.drop(['wind_profile'], axis=1)
        
        df2=self.dataframe2.groupby('paises').agg(['mean'])
        df2=df2.reset_index()
        df2.rename(columns={'paises':'id_paises'}, inplace=True)
        def minuscula(col):
            try:
                col=col.lower()
                return col
            except:
                pass
        df2["id_paises"]=df2["id_paises"].apply(minuscula)

        return df2
    
    

In [5]:
Limpieza=Limpiar_dfs(attack,df)


In [6]:
df_attack=Limpieza.limpiar_attack() #No funciona :(

/tmp/ipykernel_21541/2911249073.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attack.rename(columns={'country':'id_paises'}, inplace=True)
/tmp/ipykernel_21541/2911249073.py:14: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_final=df_attack.groupby('id_paises')['year','age','latitud','longitud','fatal_N'].agg(['mean'])


In [10]:
df_attack.head()

,id_paises,year,age,latitud,longitud,fatal_N
,,mean,mean,mean,mean,mean
0,australia,1945.205479,27.527397,-24.776109,134.755000,0.690259
1,new zealand,1941.134921,27.174603,-41.500083,172.834408,0.706349
2,papua new guinea,1914.460317,24.666667,-5.681607,144.248908,0.531746
3,south africa,1960.784965,26.159091,-28.816624,24.991639,0.709790
4,usa,1971.065544,26.842057,39.783730,-100.445882,0.821575


In [7]:
df_clima=Limpieza.limpieza_clima()

/tmp/ipykernel_21541/2911249073.py:43: FutureWarning: ['prec_type'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df2=self.dataframe2.groupby('paises').agg(['mean'])


In [8]:
df_clima

,id_paises,wind_950mb,wind_900mb,wind_850mb,wind_800mb,wind_750mb,wind_700mb,wind_650mb,wind_600mb,wind_550mb,wind_500mb,wind_450mb,wind_400mb,wind_350mb,wind_300mb,wind_250mb,wind_200mb,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,rh450mb,rh400mb,rh350mb,rh300mb,rh250mb,rh200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud
,,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
0,australia,109.843750,139.921875,161.484375,180.468750,173.984375,183.984375,205.390625,219.375000,245.390625,252.265625,248.281250,249.609375,254.609375,250.937500,248.593750,253.593750,12.765625,10.906250,5.890625,1.328125,0.406250,0.203125,-0.296875,-1.015625,-1.515625,-1.781250,-2.078125,-1.031250,0.140625,2.093750,4.875000,1.390625,97.5,3.859375,-9999.0,-9999.0,-9999.0,21.328125,4.765625,10.171875,1022.859375,2.984375,0.0,114.843750,3.140625,-24.776100,134.755000
1,new zealand,212.500000,214.296875,216.171875,226.875000,226.406250,228.203125,226.484375,228.437500,227.812500,227.812500,225.703125,217.421875,225.703125,236.640625,252.265625,262.890625,11.296875,13.375000,9.546875,3.687500,0.578125,-0.640625,-0.500000,-0.203125,-0.484375,0.296875,1.437500,0.640625,0.250000,0.671875,1.046875,-3.093750,97.5,7.781250,-9999.0,-9999.0,-9999.0,9.843750,9.468750,8.437500,1025.203125,2.593750,0.0,178.906250,3.046875,-41.500083,172.834408
2,papua new guinea,105.546875,102.031250,100.000000,99.765625,94.375000,93.828125,110.078125,114.140625,112.421875,120.781250,103.515625,197.500000,173.203125,199.843750,245.781250,279.140625,13.593750,11.890625,9.156250,6.093750,1.062500,-0.875000,-2.218750,-3.218750,-3.625000,-3.515625,-2.859375,-2.031250,-1.437500,-1.203125,-1.546875,-0.031250,97.5,2.109375,-9999.0,-9999.0,-9999.0,25.781250,1.609375,10.953125,1011.796875,2.281250,0.0,105.390625,3.703125,-5.681607,144.248908
3,south africa,201.796875,217.812500,236.640625,288.671875,280.390625,263.515625,260.312500,270.000000,279.921875,282.187500,281.718750,276.796875,271.796875,267.734375,269.531250,265.937500,12.921875,11.984375,11.312500,8.890625,8.515625,8.953125,8.875000,8.421875,7.609375,6.968750,8.484375,10.062500,10.890625,8.890625,7.265625,3.671875,97.5,8.000000,-9999.0,-9999.0,-9999.0,16.609375,3.937500,12.484375,1018.046875,5.546875,0.0,163.125000,3.203125,-28.816624,24.991639
4,usa,174.062500,172.890625,193.437500,249.765625,292.500000,291.093750,286.406250,279.218750,277.109375,277.578125,278.203125,267.109375,248.906250,244.765625,245.625000,271.171875,8.546875,9.578125,9.265625,9.015625,8.078125,5.859375,3.656250,2.906250,3.234375,3.812500,4.718750,6.281250,6.968750,8.078125,9.140625,4.203125,97.5,4.281250,-9999.0,-9999.0,-9999.0,25.140625,0.140625,6.437500,1012.187500,2.984375,0.0,175.390625,2.296875,39.783730,-100.445882


In [9]:
def merge(self): #Esta función está persanda para hacer un left. Para que se tenga en cuenta
    merge_final=pd.merge(left=self.df_left, right=self.df_rigth, how='left', left_on='id_paises', right_on='id_paises')
    return merge_final

## UNA CLASE 

In [ ]:
class concat_clean_merge():
     
    def paises(producto):

        df_concat=pd.DataFrame()
        dict={'USA':[39.783730,-100.445882],'Australia':[-24.77610,134.755], 'South Africa':[-28.8166236,24.991639],\
            'New Zealand':[-41.5000831,172.8344077],'Papua New Guinea':[-5.6816069,144.2489081] }
            
        for k,v in dict.items():
                
            url = f'http://www.7timer.info/bin/api.pl?lon=-{v[1]}&lat={v[0]}&product={producto}&output=json'

            response = requests.get(url)
            df=pd.json_normalize(response.json()["dataseries"])
            df["latitud"] = v[0]
            df["longitud"] = v[1]
            df['paises']= k
            df_concat=pd.concat([df_concat,df], axis=0, ignore_index=True)

        return df_concat

    def limpiar_attack (df):

        df_attack=df[(df["country"]=="usa") | (df["country"]=="australia")|\
            (df["country"]=="south africa") | (df["country"]=="papua new guinea")|(df["country"]=="new zealand")]

        df_attack.rename(columns={'country':'id_paises'}, inplace=True)

        df_final=df_attack.groupby('id_paises')['year','age','latitud','longitud','fatal_N'].agg(['mean'])
        df_final=df_final.reset_index()

        return df_final
    
    def limpieza_clima(df):

        #Limpiamos la columna de rh_profile primero a series y luego a columnas en el bbucle for
        df['rh_profile']=df['rh_profile'].apply(ast.literal_eval)
        rh = df['rh_profile'].apply(pd.Series)

        for i in range(len(rh.columns)):
            rh_col0=rh[i].apply(pd.Series)
            nombre="rh"+ rh_col0["layer"][115]
            valores= list(rh_col0["rh"])
            df.insert(i, nombre, valores)
            
        #Eliminamos la columna que ya no necesitamos
        df=df.drop(['rh_profile'], axis=1)
        
        #Hacemos lo mismo para la columnas wind_profile
        df['wind_profile']=df['wind_profile'].apply(ast.literal_eval)
        wind = df['wind_profile'].apply(pd.Series)

        for i in range(len(wind.columns)):
            wind_col0=wind[i].apply(pd.Series)
            nombre="wind_"+ wind_col0["layer"][1]
            valores= list(wind_col0["direction"])
            df.insert(i, nombre, valores)
            
        df=df.drop(['wind_profile'], axis=1)
        
        df2=df.groupby('paises').agg(['mean'])
        df2=df2.reset_index()
        df2.rename(columns={'paises':'id_paises'}, inplace=True)
        def minuscula(col):
            try:
                col=col.lower()
                return col
            except:
                pass
        df2["id_paises"]=df2["id_paises"].apply(minuscula)

        return df2

    def merge(df_left, df_right): #Esta función está persanda para hacer un left. Para que se tenga en cuenta
        merge_final=pd.merge(left=df_left, right=df_right, how='left', left_on='id_paises', right_on='id_paises')
        return merge_final